In [10]:
#导入各种需要的库
import geemap
import ee
Map = geemap.Map()
from IPython.display import Image
import PySimpleGUI as sg
from tqdm.notebook import tqdm
from time import sleep
import csv
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(processName)s %(message)s"
)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:


# set study area
Ukraine = ee.FeatureCollection("FAO/GAUL/2015/level0") \
                .filter(ee.Filter.eq('ADM0_NAME','Ukraine'))
roi = Ukraine
Map.addLayer(roi, {'color':'grey'}, 'studyArea')
Map.centerObject(roi)

year_start = 2020
year_end = 2020

imgCol = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR") \
              .filterBounds(roi) \
              .filter(ee.Filter.calendarRange(year_start, year_end,'year'))

# compute the image total size
imgSize = imgCol.size()
print("imgSize",imgSize.getInfo()); # result is 1016

#****************************************************************
# compute the image avaibility. There are several ways to achieve that.
# '1': use the count function
# '2': use the sum function
##################################################################
imgCount_V1 = imgCol.select("B1").count().clip(roi)

def func_sqx(img):
    return img.multiply(0).add(1).toInt()

imgCount_V2= imgCol.select("B1").map(func_sqx).sum().clip(roi)

# define image render parameters
visParam = {
 'min': 10,
 'max': 40,
 'palette': 'FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718, 74A901, 66A000, 529400,' + \
   '3E8601, 207401, 056201, 004C00, 023B01, 012E01, 011D01, 011301'
}

Map.addLayer(imgCount_V1,visParam,"imgCount_V1")
Map.addLayer(imgCount_V2,visParam,"imgCount_V2")


imgSize 1016
